In [13]:
import pandas as pd
import tqdm as tqdm
import requests

S = requests.Session()

In [7]:
data = pd.read_csv("scraped.csv")

del data['Unnamed: 0']

data['location'] = ''
data['inception'] = ''
data['image'] = ''
data['discovery'] = ''
data['country'] = ''

data.head()

,category,entity,text,title,location,inception,image,discovery,country
0,Category:Archaeological artefact types,Q2517447,An adze (; alternative spelling: adz) is a cut...,Adze,,,,,
1,Category:Archaeological artefact types,Q262026,"An andiron or firedog, fire-dog or fire dog is...",Andiron,,,,,
2,Category:Archaeological artefact types,Q1196338,"The ard, ard plough, or scratch plough is a si...",Ard (plough),,,,,
3,Category:Archaeological artefact types,Q45922,An arrow is a fin-stabilized projectile that i...,Arrow,,,,,
4,Category:Archaeological artefact types,Q220659,"An artifact, or artefact (see American and Bri...",Artifact (archaeology),,,,,


In [28]:
def scrape_page_wikidata(entity):
    PARAMS = {
        "format": "json",
        "query":
        '\
            SELECT ?wdLabel ?ps_Label ?wdpqLabel ?pq_Label {\
              VALUES (?company) {(wd:' + entity + ')}\
                    ?company ?p ?statement .\
                    ?statement ?ps ?ps_ .\
                    ?wd wikibase:claim ?p.\
                    ?wd wikibase:statementProperty ?ps.\
                    OPTIONAL {\
                      ?statement ?pq ?pq_ .\
                      ?wdpq wikibase:qualifier ?pq .\
                    }\
                  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }\
                }\
            ORDER BY ?wd ?statement ?ps_'
    }
    R = S.get(url="https://query.wikidata.org/sparql", params=PARAMS)
    try:
        wikidata_json = R.json()
        results = {}
        print(wikidata_json)
        for obj in wikidata_json['results']['bindings']:
            key = obj['wdLabel']['value']
            val = obj['ps_Label']['value']
            results[key] = val
        return results
    except Exception as e:
        print("An exception occurred at page ", entity)
        print(R.text)
        print(e)
        return {}

In [29]:
scrape_page_wikidata(data.iloc[1]['entity'])    

An exception occurred at page  Q262026
<html>
<head>
<meta http-equiv="Content-Type" content="text/html;charset=utf-8"/>
<title>Error 403 You have been banned until 2019-07-12T08:31:34.311Z, please respect throttling and retry-after headers.</title>
</head>
<body><h2>HTTP ERROR 403</h2>
<p>Problem accessing /bigdata/namespace/wdq/sparql. Reason:
<pre>    You have been banned until 2019-07-12T08:31:34.311Z, please respect throttling and retry-after headers.</pre></p><hr><a href="http://eclipse.org/jetty">Powered by Jetty:// 9.4.12.v20180830</a><hr/>

</body>
</html>

Expecting value: line 1 column 1 (char 0)


{}

In [19]:
data.iloc[1]['entity']

'Q262026'